In [ ]:
!pip install transformers==3.0.2

In [ ]:
import re

import torch
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [ ]:
token_ids = tokenizer.encode('The best movie ever!')

In [ ]:
token_ids

In [ ]:
tokenizer.decode(token_ids)

In [ ]:
!curl https://s3.amazonaws.com/realworldnlpbook/data/stanfordSentimentTreebank/trees/dev.txt --output dev.txt

In [ ]:
def read_dataset(file_path, batch_size, tokenizer, max_length):
    batches = []
    with open(file_path) as f:
        texts = []
        labels = []
        for line in f:
            text = line.strip()
            label = int(text[1])
            text = re.sub('\)+', '', re.sub('\(\d ', '', text))
            text = text.replace('-LRB-', '(').replace('-RRB-', ')')
            
            texts.append(text)
            labels.append(label)

            if len(texts) == batch_size:
                batch = tokenizer(
                    texts,
                    max_length=max_length,
                    pad_to_max_length=True,
                    truncation=True,
                    return_tensors='pt')
                batch['labels'] = torch.tensor(labels, dtype=torch.int32)
                batches.append(batch)
                
                texts = []
                labels = []
        
        if texts:
            batch = tokenizer(
                texts,
                max_length=max_length,
                pad_to_max_length=True,
                truncation=True,
                return_tensors='pt')
            batch['labels'] = torch.tensor(labels)
            batches.append(batch)

        return batches

In [ ]:
read_dataset('dev.txt', batch_size=32, tokenizer=tokenizer, max_length=128)